In [1]:
from os import system
import pickle

import numpy as np

## PFLOTRAN
import jupypft.model as mo
import jupypft.parameter as pm
import jupypft.plotBTC as plotBTC

def resetPickle():
    mo.Model.resetListOfModels()
    global caseDict
    with open('caseDict.pkl', 'rb') as f:
        caseDict = pickle.load(f)
    
    pm.Parameter.rebuildListOfObjects(caseDict)
    
    global listOfAllParameters
    listOfAllParameters = pm.Parameter.list_of_vars()
    
    #system("rm -rf CASE*")
    
def plotResults(clean=True,BTC=False,EndC=False):
    if clean:
        system("rm -rf MASSBALANCES; mkdir MASSBALANCES")
        system("cp ./CASE**/*-mas.dat ./MASSBALANCES")
        mo.Model.folderFixedToCSV("MASSBALANCES")

    waterDensity = 999.65
    m3ToL = 1000.

    if BTC:
        plotBTC.plotMassBalancesInFolder(
            folderToPlot="MASSBALANCES",
            indices = {'t':"Time [d]",\
                       'q':"ExtractWell Water Mass [kg/d]",\
                       'm':"ExtractWell Vaq [mol/d]"},
            normalizeWith={'t':1.0,'q':waterDensity/m3ToL,'m':1.0},
            legendTitle = legendTitle)

    if EndC:
        plotBTC.plotEndConcentrations(
            folderToPlot="MASSBALANCES",
            Xdata = Iarray,
            indices = {'t':"Time [d]",\
                       'q':"ExtractWell Water Mass [kg/d]",\
                       'm':"ExtractWell Vaq [mol/d]"},
            normalizeWith={'t':1.0,'q':waterDensity/m3ToL,'m':1.0},
            legendTitle = legendTitle)
    
def buildSim(caseName):
    ## Create a folder for the case
    currentFolder = "./CASE_{0}".format(caseName)
    currentFile = currentFolder + "/" + caseName +".in"
    system("mkdir " + currentFolder)
    
    ## Initialize PFLOTRAN model
    BoxModel = mo.Model(
        templateFile = templateFile,
        runFile = currentFile,
        execPath = execPath,
        verbose=True
        )
       
    ## Copy template input file to folder
    BoxModel.cloneTemplate()
    
    ## Replace tags for values in case
    for parameter in listOfAllParameters:
        BoxModel.replaceTagInFile(parameter)

    return BoxModel

## Dummy for caseDict
caseDict = {}

def getTemplate(key):
    templateFiles = {"TH_RSandbox_Const":"tpl_TH_3Dbox_bioparticleKte_NoH5.in"}

    templateFolder = "../TEMPLATES/boxes_3D/"
    return templateFolder + templateFiles[key]
    
templateFile = getTemplate('TH_RSandbox_Const')
execPath = "$PFLOTRAN_DIR/buildExperimental/pflotran"

In [2]:
K = 10.**-2.
Qin = 0.24
f = 10.
H = 20.
r = 40.
I = 0.001
C0 = 1.0
decayRate = 3.5353E-06
attachRate = 7.2E-06

nu = 0.0000013081 #m²/s
g = 9.81 #m/s²
THETA = 0.35

In [3]:
resetPickle()
caseDict['endTime'].value = 50.

caseDict['BIOPARTICLE']['decayAq'].value = decayRate
caseDict['BIOPARTICLE']['katt'].value = attachRate

caseDict['Q']['In'].value  = Qin
caseDict['Q']['Out'].value = -Qin*f
caseDict['k']['X'].value = K*nu/g
caseDict['k']['Y'].value = K*nu/g
caseDict['k']['Z'].value = K*nu/g
caseDict['theta'].value = THETA
caseDict['L']['Z'].value = H
caseDict['inCoord']['X'][1].value = caseDict['outCoord']['X'][1].value + r
caseDict['inCoord']['X'][2].value = caseDict['outCoord']['X'][2].value + r

caseDict['inCoord']['Z'][1].value = 0.0
caseDict['inCoord']['Z'][2].value = H

caseDict['outCoord']['Z'][1].value = 0.0
caseDict['outCoord']['Z'][2].value = H

In [4]:
Qin_array = np.array([0.24,1.,10.,100.])
r_array = np.array([5,10,40,100])
I_array = np.array([[0.07,0.13,0.48,1.20],
                    [0.07,0.13,0.48,1.20],
                    [0.08,0.13,0.48,1.20],
                    [0.17,0.21,0.53,1.20]])/100.

In [5]:
i,j = 0,0
ref = 3
for qi,Qin in enumerate(Qin_array):
    caseDict['Q']['In'].value  = Qin
    for ri,r in enumerate(r_array):
        caseDict['inCoord']['X'][1].value = caseDict['outCoord']['X'][1].value + r
        caseDict['inCoord']['X'][2].value = caseDict['outCoord']['X'][2].value + r
        Icentral = I_array[qi,ri]
        I_guess_1 = np.linspace(Icentral/3.,Icentral,num=ref)
        I_guess_2 = np.linspace(Icentral,Icentral*3.,num=ref)
        I_guess = np.concatenate((I_guess_1,I_guess_2[1:]))
        for ii, I in enumerate(I_guess):
            caseDict['Gradient']['X'].value  = I
            buildSim("{:02d}_{:02d}__I_=_{:.1E}".format(i,j,I))
            j += 1
        j = 0
        i += 1

In [6]:
## Run and plot
mo.Model.runAllModels(nProcs=8)
#legendTitle = "Decay rates $\lambda$  [1/s]"
#plotResults()

In [8]:
waterDensity = 999.65
m3ToL = 1000.
import os 
from pandas import read_csv

folderToPlot="Comparison_MASSBALANCES_att"

#os.system("rm -rf MASSBALANCES; mkdir MASSBALANCES")
#os.system("cp ./CASE**/*-mas.dat ./MASSBALANCES")

mo.Model.folderFixedToCSV(folderToPlot)     

indices = {'t':"Time [d]",\
           'q':"ExtractWell Water Mass [kg/d]",\
           'm':"ExtractWell Vaq [mol/d]"}
normalizeWith={'t':1.0,'q':waterDensity/m3ToL,'m':1.0}

listOfFiles = os.listdir(folderToPlot)
listOfFiles.sort()
      
logC_arr = np.zeros_like(listOfFiles)
    
for i,f in enumerate(listOfFiles):
    DATA = read_csv(\
    folderToPlot+"/"+f,\
    delimiter=",")

    q = DATA[indices['q']]/normalizeWith['q']
    m = DATA[indices['m']]/normalizeWith['m']
      
    Y = np.divide(m,q)
    maxY = np.max(Y)
    
    logC_val = -np.log10(maxY)
    logC_arr[i] = logC_val

NumExpr defaulting to 8 threads.


In [9]:
i,j = 0,0
ref = 3

I = []
for qi,Qin in enumerate(Qin_array):
    for ri,r in enumerate(r_array):
        Icentral = I_array[qi,ri]
        I_guess_1 = np.linspace(Icentral/3.,Icentral,num=ref)
        I_guess_2 = np.linspace(Icentral,Icentral*3.,num=ref)
        I_guess = np.concatenate((I_guess_1,I_guess_2[1:]))
        I.append(I_guess)
        
minC_array = np.zeros(16)
minI_array = np.zeros(16)

for i,vect in enumerate(np.reshape(logC_arr,(16,5))):
    Ieach = I[i]
    minC_array[i] = min(vect)
    minI_array[i] = Ieach[np.argmin(vect)]

In [10]:
minI_array

array([0.0014, 0.0039, 0.0144, 0.036 , 0.0014, 0.0039, 0.0144, 0.036 ,
       0.0016, 0.0039, 0.0144, 0.036 , 0.0017, 0.0063, 0.0159, 0.036 ])

In [11]:
minC_array

array([2.70966103, 3.09070854, 3.61937381, 3.99433636, 2.0934136 ,
       2.47294713, 3.00034689, 3.37487213, 1.13490091, 1.49712873,
       2.00931935, 2.37863042, 0.39427172, 0.66484608, 1.08376174,
       1.41433393])